In [ ]:
pip install crewai crewai-tools langchain_groq

In [ ]:
from crewai import Agent, Task, Crew, Process
from langchain.tools import tool
from crewai.tasks.task_output import TaskOutput
import json, requests, os
import os
import dotenv
dotenv.load_dotenv()
from langchain_groq import ChatGroq


In [ ]:
llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0, api_key=os.environ["GROQ_API_KEY"])

In [ ]:
class JobSearchTools:
    @tool("Job Search Tool")
    def search_jobs(input_json: str) -> str:
        """Search for job openings using the Adzuna API."""
        try:
            input_data = json.loads(input_json)
            role = input_data['role']
            location = input_data['location']
            num_results = input_data['num_results']
        except (json.JSONDecodeError, KeyError) as e:
            return """The tool accepts input in JSON format with the
                    following schema: {'role': '<role>', 'location': '<location>', 'num_results': <number>}.
                    Ensure to format the input accordingly."""

        app_id = os.getenv('ADZUNA_APP_ID')
        api_key = os.getenv('ADZUNA_API_KEY')
        base_url = "http://api.adzuna.com/v1/api/jobs"
        url = f"{base_url}/us/search/1?app_id={app_id}&app_key={api_key}&results_per_page={num_results}&what={role}&where={location}&content-type=application/json"

        try:
            response = requests.get(url)
            response.raise_for_status()
            jobs_data = response.json()

            job_listings = []
            for job in jobs_data.get('results', []):
                job_details = f"Title: {job['title']}, Company: {job['company']['display_name']}, Location: {job['location']['display_name']}, Job URL:{job['redirect_url']}, Description: {job['description'][:100]}..."
                job_listings.append(job_details)
            return '\n'.join(job_listings)
        except requests.exceptions.HTTPError as err:
            raise ToolException(f"HTTP Error: {err}")
        except requests.exceptions.RequestException as e:
            raise ToolException(f"Error: {e}")

In [ ]:
job_searcher_agent = Agent(
    role='Job Searcher',
    goal='Find and secure job opportunities that align with your resume',
    backstory="""
    You are a proactive job seeker, focused on finding roles that match your experience and skills.
    With a strategic approach to job searching, you aim to secure positions that offer both professional growth and opportunities to expand your expertise.
    """,
    verbose=True,
    llm=llm,
    allow_delegation=True,
    tools=[JobSearchTools().search_jobs]
)

In [ ]:
interview_preparation_agent = Agent(
    role='Interview Preparation Coach',
    goal='Enhance interview skills, focusing on common questions',
    backstory="""Expert in coaching job searchers on successful interview techniques, including mock interviews and feedback.""",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

In [ ]:
job_search_task = Task(
    description="""Search for current job openings for the Data Scientist role in New York
    using the Job Search tool. Find 5 vacant positions in total.
    Emphasize the key skills required.
    The tool accepts input in JSON format with the
    following schema: {\"role\": \"Data Scientist\", \"location\": \"New York\", \"num_results\": 5}.
    Ensure to format the input accordingly.""",
    expected_output="""
    A list containing details of 5  Data Scientist job openings in New York, It should contain, Job title, Company name, location, job url and
    description including the key skills required for each position.""",
    agent=job_searcher_agent,
    tools=[JobSearchTools().search_jobs],
    output_file = 'job info.txt'

)


In [ ]:
interview_preparation_task = Task(
    description="""Prepare job searchers for interviews by conducting mock interview for each role separately.
    Prepare the questions from the Job role, description and skills""",
    expected_output = """ List containing the details of job along with all the information and skills and also 5 interview questions for each role """,
    agent=interview_preparation_agent,
    context=[job_search_task ],
    output_file = 'interview_questions.txt'
)

In [ ]:
job_search_crew = Crew(
    agents=[job_searcher_agent, interview_preparation_agent],
    tasks=[job_search_task,interview_preparation_task ],
    process=Process.hierarchical,
    manager_llm=llm,
)

In [ ]:
crew_result = job_search_crew.kickoff()

print(crew_result)



> Entering new CrewAgentExecutor chain...
Thought: I need to find current job openings for the Data Scientist role in New York using the Job Search tool.

Action: Job Search Tool
Action Input: {"role": "Data Scientist", "location": "New York", "num_results": 5} 

I encountered an error while trying to use the tool. This was the error: 1 validation error for Job Search ToolSchema
input_json
  field required (type=value_error.missing).
 Tool Job Search Tool accepts these inputs: Search for job openings using the Adzuna API.

Thought: I need to find current job openings for the Data Scientist role in New York using the Job Search tool.

Action: Job Search Tool
Action Input: {"input_json": "{\"role\": \"Data Scientist\", \"location\": \"New York\", \"num_results\": 5}"} 

Title: Data Scientist, Company: Robert Half, Location: Grand Central, Manhattan, Job URL:https://www.adzuna.com/details/4755387491?utm_medium=api&utm_source=13fdb047, Description: Description We are on the hunt for a Da